In [1]:
import numpy as np

#storage
import shelve
import zlib

# filesistem
import tempfile
import os

# pretty printing
from pprint import pformat
from IPython.display import Markdown

# multiprocessing
from multiprocessing import Pool, cpu_count

# Esegue un file .wip

In [2]:
file_name = "./simulation_data/square_hq.wip"
workers = cpu_count()

Otteniamo quanto abbiamo ancora da fare, estraendo l'index e eliminando le simulazioni già fatte

In [3]:
with shelve.open(file_name, "r") as wip_file:
    simulations_index = wip_file[".index"]
total_sims = len(simulations_index)
# filtering out already done simulations
simulations_index={simulation:simulations_index[simulation] for simulation in simulations_index if simulations_index[simulation]['status'] != "done"}
print(f"Wip file is {100 * (1-len(simulations_index)/total_sims)}% done")

Wip file is 0.0% done


Per eseguire una simulazione devo estrarre il simulatore e lanciarlo con i parametri dati, catturando lo stdout

In [4]:
with shelve.open(file_name, "r") as wip_file:
    invocation_string = wip_file[".invocation_string"]

def run_simulation(simulator_file_name, setup):
    # running the simulator
    result = !{simulator_file_name} {invocation_string.format(**setup)}
    # splitting results and converting to float
    result = np.array([
        [float(val) for val in line.split(" ")] 
        for line in result
    ])
    result = {
        'acceptance': np.mean(result[:,0]),
        'measures': result[:,1:]
    }
    return result

Eseguendo ogni simulazione ancora da fare

In [5]:
with tempfile.TemporaryDirectory() as run_dir:
    with Pool(workers) as pool:
        future_results = {}
        # loading calls into the pool
        for simulation in simulations_index:
            simulations_index[simulation]['status'] = 'running'
            with shelve.open(file_name, "w") as wip_file:
                # marking the simulation as running
                wip_file[".index"] = simulations_index
                # preparing simulator file
                simulator_file_name = os.path.join(run_dir, simulations_index[simulation]['simulator_name'])
                if not os.path.exists(simulator_file_name):
                    with open(simulator_file_name, "wb") as simulator_file:
                        simulator_file.write(zlib.decompress(wip_file[simulations_index[simulation]['simulator_name']]))
                !chmod +x {simulator_file_name}
            print(f"Running '{simulation}'")
            # running the simulation
            future_results[simulation] = pool.apply_async(run_simulation, (
                simulator_file_name, 
                simulations_index[simulation]['setup']
            ))
        # reading results back
        for simulation in simulations_index:
            result = future_results[simulation].get()
            print(f"'{simulation}' is done.")
            # shelving result
            with shelve.open(file_name, "w") as wip_file:
                wip_file[simulation] = result
                simulations_index[simulation]['status'] = 'done'
                wip_file[".index"] = simulations_index
            del future_results[simulation]

Running 'Fwml7t_k_DTpo3TDwhorziQiCfyWqphsCOfBAQ==.subsim'
Running 'Ki_tpF8h0Xxw3bQ4B84vahhZWtbeT3pl2U2AfA==.subsim'
Running 'vsJ39X48T5vwe8u10DokGSedChB0F-DZHEIJhw==.subsim'
Running 'gc5deWrf4zhuu4878OBc2qw3pMDwoDJhnGnOtQ==.subsim'
Running 'shWRCE21vKg_xHqJa5Z1Ma65N31euEz8Kzz6Gg==.subsim'
'Fwml7t_k_DTpo3TDwhorziQiCfyWqphsCOfBAQ==.subsim' is done.
'Ki_tpF8h0Xxw3bQ4B84vahhZWtbeT3pl2U2AfA==.subsim' is done.
'vsJ39X48T5vwe8u10DokGSedChB0F-DZHEIJhw==.subsim' is done.
'gc5deWrf4zhuu4878OBc2qw3pMDwoDJhnGnOtQ==.subsim' is done.
'shWRCE21vKg_xHqJa5Z1Ma65N31euEz8Kzz6Gg==.subsim' is done.


In [6]:
report = "### Content of the wip file:\n"
with shelve.open(file_name, "r") as wip_file:
    for item in wip_file:
        report += f"#### {item}:\n    "
        if item.endswith(".simulator"):
            report += "<binary data>\n"
        else:
            report += pformat(wip_file[item], depth=2).replace("\n", "\n    ") + "\n"
Markdown(report)

### Content of the wip file:
#### .setup:
    {'V_tilde': ['if(y - floor(y) < 0.5) return 0; else return -1;'],
     'beta_tilde': [300],
     'd': [0.1, 2.575, 5.05, 7.525, 10.0],
     'eta': [0.1],
     'local_steps': [5],
     'measuration_to_take': [500],
     'measure_every': [1000],
     'seed': [42],
     'step_size': [0.4]}
#### .simulate_hash:
    'd95d2ad1326601dce16c7c311803bf8dfa9a3c0af7b69f1b543e52b3'
#### .invocation_string:
    ('{seed} {d} {eta} {N} {local_steps} {measuration_to_take} {measure_every} '
     '{step_size}')
#### shWRCE21vKg_xHqJa5Z1Ma65N31euEz8Kzz6Gg==.subsim:
    {'acceptance': 0.463831774,
     'measures': array([[ 0.605191,  0.739774,  0.67191 , ...,  0.556328,  0.602865,
             0.754756],
           [ 0.627657,  0.524662,  0.320106, ...,  0.595322,  0.393966,
             0.557206],
           [ 0.841865,  1.015168,  0.98873 , ...,  1.433009,  1.409292,
             0.929053],
           ...,
           [-1.501025, -1.487831, -1.508746, ..., -1.460321, -1.500891,
            -1.660976],
           [-2.11452 , -2.095851, -2.086022, ..., -2.0891  , -2.034467,
            -1.957113],
           [-1.296853, -1.322768, -1.035227, ..., -1.414119, -1.395813,
            -1.424403]])}
#### .index:
    {'Fwml7t_k_DTpo3TDwhorziQiCfyWqphsCOfBAQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'Ki_tpF8h0Xxw3bQ4B84vahhZWtbeT3pl2U2AfA==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'gc5deWrf4zhuu4878OBc2qw3pMDwoDJhnGnOtQ==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'shWRCE21vKg_xHqJa5Z1Ma65N31euEz8Kzz6Gg==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'},
     'vsJ39X48T5vwe8u10DokGSedChB0F-DZHEIJhw==.subsim': {'setup': {...},
                                                         'simulator_name': 'i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator',
                                                         'status': 'done'}}
#### vsJ39X48T5vwe8u10DokGSedChB0F-DZHEIJhw==.subsim:
    {'acceptance': 0.528229012,
     'measures': array([[ 0.742469,  0.654279,  0.631659, ...,  0.974946,  0.913329,
             0.666843],
           [ 1.924199,  1.840366,  1.916606, ...,  1.809291,  1.778121,
             1.952068],
           [ 0.812877,  0.776929,  0.64454 , ...,  0.722288,  0.697084,
             0.555569],
           ...,
           [-5.621152, -5.528347, -5.2307  , ..., -5.851155, -5.652142,
            -5.91879 ],
           [-4.062441, -4.287796, -4.271916, ..., -4.314571, -4.511031,
            -4.134243],
           [-4.123238, -4.399217, -4.489193, ..., -4.158009, -3.743749,
            -4.012985]])}
#### Ki_tpF8h0Xxw3bQ4B84vahhZWtbeT3pl2U2AfA==.subsim:
    {'acceptance': 0.588254246,
     'measures': array([[-1.020139, -0.796076, -0.491765, ..., -0.253   , -0.078986,
            -0.309121],
           [-0.214268, -0.172321,  0.030704, ...,  0.298179,  0.022613,
            -0.252965],
           [ 2.609717,  2.623973,  2.620476, ...,  2.653002,  2.773114,
             2.882385],
           ...,
           [-2.988055, -2.891322, -2.859329, ..., -2.572028, -2.996292,
            -3.075345],
           [-5.237595, -5.412798, -5.291681, ..., -4.551598, -4.637732,
            -4.765111],
           [-5.812358, -5.774146, -5.253997, ..., -5.641038, -5.748207,
            -5.50788 ]])}
#### Fwml7t_k_DTpo3TDwhorziQiCfyWqphsCOfBAQ==.subsim:
    {'acceptance': 0.8033215600000001,
     'measures': array([[ 2.092304,  2.032908,  2.827905, ...,  2.809293,  2.445684,
             2.120698],
           [-0.485275, -0.25306 , -0.202358, ..., -0.295805, -0.509103,
            -0.151689],
           [ 2.153258,  1.916985,  1.532603, ...,  2.471498,  2.11359 ,
             2.249791],
           ...,
           [-2.43946 , -2.444654, -2.214492, ..., -3.394366, -2.819285,
            -2.09228 ],
           [-2.512914, -2.953369, -3.281913, ..., -2.1516  , -2.514899,
            -2.569653],
           [-0.413016, -1.01945 , -0.989828, ..., -1.416382, -1.856446,
            -1.802167]])}
#### gc5deWrf4zhuu4878OBc2qw3pMDwoDJhnGnOtQ==.subsim:
    {'acceptance': 0.490911876,
     'measures': array([[ 0.804052,  0.415964,  0.434793, ...,  0.069611,  0.268304,
             0.611543],
           [ 1.675967,  1.464543,  1.535351, ...,  1.86108 ,  1.877554,
             1.563519],
           [ 1.648108,  1.735885,  1.762707, ...,  1.215407,  1.360462,
             1.586812],
           ...,
           [-5.612448, -5.780204, -5.802386, ..., -5.035999, -4.955629,
            -5.301303],
           [-4.517921, -4.908886, -5.078765, ..., -4.289242, -4.265615,
            -4.419617],
           [-5.973516, -5.943376, -5.616268, ..., -5.779848, -5.98713 ,
            -5.897481]])}
#### i233F04XhpGVzRA4DpUioRxyGoxYKsILoBWJig==.simulator:
    <binary data>
